In [35]:
from dotenv import load_dotenv
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
index_name = "incometax-markdown"
db = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")
retriever = db.as_retriever(search_kwargs={"k": 4})

query_dict = ["사람을 나타내는 표현 -> 거주자"]

prompt_dict = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해 주세요.
만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
그런경우에는 질문만 반환해주세요.
사전: {query_dict}

질문: {{question}}
""")

dict_chain = prompt_dict | llm | StrOutputParser()

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(),
                                       chain_type_kwargs={"prompt": prompt},
                                       )
tax_chain = {"query": dict_chain} | qa_chain | StrOutputParser()

True